# Projeto 1 - Ciência dos Dados

Nome: __Eduardo Selber Castanho___

Nome: __Henrique Fazio Badin___

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [12]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [13]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\eduse\Downloads\22-2a-cd-p1-grupo_henriquefb3\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [14]:
filename = 'dados_com target (1).xlsx'

In [17]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
dados=train
train.head(5)

,Titulo,Descrição,Pagina,Target,Unnamed: 4,Unnamed: 5
0,CoinFLEX demite funcionários em meio a disputa...,A conturbada corretora de criptomoedas CoinFLE...,12,1,NaN,283.0
1,O que dar de presente neste Natal para alguém ...,Está em busca de um presente legal para um ent...,83,0,NaN,NaN
2,Direto de El Salvador: duas perguntas para o m...,O maximalista de bitcoin Max Keiser abriu as a...,91,0,NaN,NaN
3,Lamborghini de R$ 630 mil do falso Rei do Bitc...,Começa na próxima sexta-feira (22) o leilão de...,15,0,NaN,NaN
4,Nubank usou R$ 140 milhões para comprar até 10...,"Com R$ 14 bilhões em caixa, conforme o último ...",35,1,NaN,NaN


In [18]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Titulo,Descrição,Pagina,Target
0,O estranho caso dos blocos ‘vazios’ na blockch...,Um evento incomum aconteceu na rede do Bitcoin...,57,0.0
1,Alívio do bitcoin sugere que pior fase do inve...,"Após um mês de consolidação, o preço do bitcoi...",13,NaN
2,Lei que proíbe pagamentos com bitcoin na Rússi...,"O Parlamento da Rússia, conhecido como Duma, a...",27,0.0
3,Manhã Cripto: Projeto de Lei que regula cripto...,Com a regulação dos criptoativos no foco de in...,30,1.0
4,A semana das criptomoedas: ethereum (ETH) em d...,Embora muitas criptomoedas tenham caído nos úl...,16,1.0


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

O assunto selcionado foi as criptomoedas, mas especificamente a bitcoin e essas notícias foram classificadas seguindo o critério de como elas afetam diretamente o valor dela, sendo as relevantes as que mencionam isso. 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [19]:
texto=''
for i in range(0,540):
    texto+=dados['Titulo'][i]+' '

In [20]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
texto_f=cleanup(texto)

In [21]:
dados_relevantes=dados.loc[dados['Target']==1,:]
texto_relevantes=''
for i in dados_relevantes['Titulo']:
    texto_relevantes+=i+' '
texto_relevantes_f=cleanup(texto_relevantes)
dados_irelevantes=dados.loc[dados['Target']==0,:]
texto_irelevantes=''
for i in dados_irelevantes['Titulo']:
    texto_irelevantes+=i+' '
texto_irelevantes_f=cleanup(texto_irelevantes)
todas_dados_relevantes=texto_relevantes_f.split()
serie_relevante = pd.Series(todas_dados_relevantes)
todas_dados_irelevantes=texto_irelevantes_f.split()
serie_irelevante = pd.Series(todas_dados_irelevantes)
tabela_relevante=serie_relevante.value_counts()
tabela_irelevante=serie_irelevante.value_counts()
tabela_relevante_rel=serie_relevante.value_counts(True)
tabela_irelevante_rel=serie_irelevante.value_counts(True)
todas_texto=texto_f.split()
serie_todas=pd.Series(todas_texto)
tabela_todas=serie_todas.value_counts()
tabela_todas_rel=serie_todas.value_counts(True)


In [22]:
def matrizes(dados):
    i=0
    lista=[]
    while i<len(dados):
        lista.append(dados[i])
        i+=1
    matriz=[] 
    for i in lista:
        frase_ma=cleanup(i)
        frase_matriz=frase_ma.split()
        matriz.append(frase_matriz)
    return matriz
matriz=matrizes(dados['Titulo'])

In [23]:
def go20(frase,matrizl):
    frase_2=cleanup(frase)
    frase_f=frase_2.split()
    probFraseDadoR=1
    x=0
    y=0
    probFraseDadoI=1
    lista=[]
    tanafrase=True
    tanafrasei=True
    for i in frase_f:
        if i not in tabela_relevante:
            tanafrase=False
            
        if i not in tabela_irelevante:
            tanafrasei=False
        
    for i in frase_f:
        
        for c in matrizl:
            if i in c:
                
                lista.append(c)
               
    res = []
    track = []
    count = 0
  
    for sub in lista:
        res.append([]);
        for ele in sub:
            if ele not in track:
                res[count].append(ele)
                track.append(ele)
    c=0
    for number in res:
        c+=len(number)

    for word in frase_f:
        if not tanafrase:
            if word in tabela_relevante_rel:
                x=(tabela_relevante_rel[word]*len(todas_dados_relevantes))+1
                probFraseDadoR=probFraseDadoR*(x/(len(todas_texto)+c))
            else:
                probFraseDadoR=probFraseDadoR*(1/(len(todas_texto)+c))
        if not tanafrasei:
            if word in tabela_irelevante_rel:
                y=(tabela_irelevante_rel[word]*len(todas_dados_irelevantes))+1
                probFraseDadoI=probFraseDadoI*(y/(len(todas_texto)+c))
            else:
                probFraseDadoI=probFraseDadoI*(1/(len(todas_texto)+c))
            
        if tanafrasei:
            probFraseDadoI=probFraseDadoI*tabela_irelevante_rel[word]
    
        if tanafrase:
            probFraseDadoR=probFraseDadoR*tabela_relevante_rel[word]

    if probFraseDadoR>probFraseDadoI:
        return 'relevante'
    else:
        return 'Irrelevante'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [26]:
teste = pd.read_excel(filename, sheet_name = 'Teste')
dados2=teste

In [34]:

matriz2=matrizes(dados2['Titulo'])
listax=[]

dados2['Target teste']=dados2['Target']

resultado=''
def montandotargets(matriz2,dados2):
    c=0
    for i in matriz2:
        frase_teste=''
        for letra in i:
            frase_teste+=letra+' '
        resultado=go20(frase_teste,matriz2)

        if resultado=='relevante':
            listax.append(1)
            dados2['Target teste'][c]=1

        elif resultado=='Irrelevante':
            dados2['Target teste'][c]=0
            listax.append(0)
        c+=1
    return dados2
dados2=montandotargets(matriz2,dados2)

C:\Users\eduse\AppData\Local\Temp/ipykernel_6240/1468690124.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados2['Target teste'][c]=0
C:\Users\eduse\AppData\Local\Temp/ipykernel_6240/1468690124.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados2['Target teste'][c]=1


In [35]:
dados2.head(5)

,Titulo,Descrição,Pagina,Target,Target teste
0,O estranho caso dos blocos ‘vazios’ na blockch...,Um evento incomum aconteceu na rede do Bitcoin...,57,0.0,0.0
1,Alívio do bitcoin sugere que pior fase do inve...,"Após um mês de consolidação, o preço do bitcoi...",13,NaN,1.0
2,Lei que proíbe pagamentos com bitcoin na Rússi...,"O Parlamento da Rússia, conhecido como Duma, a...",27,0.0,1.0
3,Manhã Cripto: Projeto de Lei que regula cripto...,Com a regulação dos criptoativos no foco de in...,30,1.0,1.0
4,A semana das criptomoedas: ethereum (ETH) em d...,Embora muitas criptomoedas tenham caído nos úl...,16,1.0,1.0


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [30]:
def testqualidade(tabela_dados):
    count=0
    certas=0
    while count<len(tabela_dados):
        if tabela_dados['Target'][count]==tabela_dados['Target teste'][count]:
            certas+=1
        count+=1
    print('A porcentagem de acerto da previsão do código é de: {:.2f}%'.format((certas/count)*100))

In [31]:
quali_teste=testqualidade(dados2)
dados=go20(mat)


A porcentagem de acerto da previsão do código é de: 65.83%


___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**